<a href="https://colab.research.google.com/github/carlosegn/proyecciones-ventas-alimenticias/blob/main/Proyeccion_ventas_alimenticias_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CODING DOJO: Proyecto de proyección de ventas alimenticias**
## Creado por: Carlos Guerrero

* Fecha creado: 17/03/2023
* Fecha actualización: 19/03/2023
* Descripción: Proyección de ventas para productos alimenticios vendidos en diversas tiendas, análisis con Machine Learning.

In [27]:
# Importamos librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='text')

In [2]:
archivo = '/content/drive/MyDrive/CodingDojo/sales_predictions.csv'
df = pd.read_csv(archivo)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Exploración y limpieza de datos

In [3]:
# Limpieza de duplicados
df.duplicated().sum()

0

No existen valores duplicados

In [4]:
# Información de la base
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Se observan varios registros faltantes en las variables de peso del producto (Item_Weight) y tamaño de la tienda (Outlet_Size)

In [5]:
# Analizar variable Outlet_Size
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [6]:
num_missing = df['Outlet_Size'].isna().sum()

total_rows = df.shape[0]

percent_missing = (num_missing / total_rows)*100
print(f'{percent_missing:.2f}% de los valores son nulos')

28.28% de los valores son nulos


Observamos que casi el 30% de los valores de la variable Outlet_Size son nulos, y es una variable categórica que sólo tiene 3 valores (Medium, Small o High) por lo que imputar un valor por frecuencia o con una constante ingresaría muchos errores a nuestro análisis, por lo que lo mejor es eliminar dicha columna.

In [7]:
df = df.drop(columns='Outlet_Size')
df.head(1)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.3,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.138


In [8]:
# Análisis de la variable Item_Weight
# Relación entre Item_Weight e Item_Identifier
df.groupby(['Item_Identifier'])['Item_Weight'].value_counts()

Item_Identifier  Item_Weight
DRA12            11.600         6
DRA24            19.350         5
DRA59            8.270          6
DRB01            7.390          2
DRB13            6.115          5
                               ..
NCZ30            6.590          6
NCZ41            19.850         5
NCZ42            10.500         5
NCZ53            9.600          4
NCZ54            14.650         5
Name: Item_Weight, Length: 1555, dtype: int64

In [9]:
df['Item_Identifier'].nunique()

1559

De acuerdo a los datos, todos los Item_Identifier se corresponden con un Item_Weight, por lo que podríamos imputar los valores faltantes en Item_Weight consultando los valores de Item_Identifier e imputando un valor conocido que se obtenga de otro registro, existen 1555 registros de Item_Weight agrupados por Item_Identifier vs 1559 registro de Item_Idenfier, lo que nos quiere decir que hay 4 Items que no tienen peso registrado.

In [10]:
# Obtener los Item_Identifier con valores de Item_Weight NaN
filtro_item_weight = df['Item_Weight'].isna()
df[filtro_item_weight]['Item_Identifier']

7       FDP10
18      DRI11
21      FDW12
23      FDC37
29      FDC14
        ...  
8485    DRK37
8487    DRG13
8488    NCN14
8490    FDU44
8504    NCN18
Name: Item_Identifier, Length: 1463, dtype: object

In [11]:
# Iteramos por la serie de valores que tienen un valor NaN en Item_Weight
for index, value in df[filtro_item_weight]['Item_Identifier'].iteritems():
  
  # Creamos un filtro para cada valor de Item_Identifier
  filtro_id = df['Item_Identifier'] == value

  # Guardamos el peso máximo para ese Item_Identifier, funciona con mean() o min() también ya que son valores únicos
  peso = df[filtro_id]['Item_Weight'].max()

  # Cambiamos el valor en el indice obtenido de la serie y en el atributo Item_Weight
  df.at[index, 'Item_Weight'] = peso

In [12]:
# Confirmamos si quedan valores nulos
df.isna().sum()

Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

Confirmamos que quedan sólo 4 registros con valores nulos lo cual confirma el análisis anterior. A estos 4 registros podemos imputar un valor de media con SimpleImputer.

In [13]:
# Revisar categorías en valores Item_Fat_Content
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [14]:
df['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace=True)
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

## Preparar los datos para aprendizaje automático

In [15]:
# El objetivo es Item_Outlet_Sales
y = df['Item_Outlet_Sales']

# En las características quitaremos las columnas de Item_Identifier y Outlet_Identifier 
# ya que sólo son identificadores de los items y las tiendas y no ayudan al análisis de predicción de ventas
X = df.drop(columns=['Item_Outlet_Sales', 'Item_Identifier', 'Outlet_Identifier'])

In [16]:
# Realizamos un Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6392 entries, 4776 to 7270
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                6390 non-null   float64
 1   Item_Fat_Content           6392 non-null   object 
 2   Item_Visibility            6392 non-null   float64
 3   Item_Type                  6392 non-null   object 
 4   Item_MRP                   6392 non-null   float64
 5   Outlet_Establishment_Year  6392 non-null   int64  
 6   Outlet_Location_Type       6392 non-null   object 
 7   Outlet_Type                6392 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 449.4+ KB


In [18]:
# Selectores de columnas numéricas y categóricas
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [19]:
# Instanciar SimpleImputer
mean_imputer = SimpleImputer(strategy='mean')

# Instanciar Scaler
scaler = StandardScaler()

# Instanciar OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [28]:
# Instanciar pipeline para valores numéricos
# Primero imputamos valores con mean_imputer y luego escalamos con scaler
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [29]:
# Instanciar pipeline para valores nominales (categóricos)
# Solo codificamos con OneHotEncoder no tenemos valores a imputar
categorical_pipe = make_pipeline(ohe)
categorical_pipe

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [30]:
# Creamos tuplas que seran utilizadas por ColumnTransformer
# Cada tupla tiene el pipeline y el respectivo selector
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

# Instanciamos ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fece83cadc0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fece8461850>)])

In [31]:
# Ajustamos con los valores de entrenamiento
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fece83cadc0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fece8461850>)])

In [32]:
# Transformamos los datos de entrenamiento y prueba
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [33]:
X_train_processed

array([[ 0.73647351, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.49910838, -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.1266724 ,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.0062066 , -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.59961942, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.73647351, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])

In [34]:
X_test_processed

array([[ 0.29411123, -0.77664625, -0.99881554, ...,  1.        ,
         0.        ,  0.        ],
       [-1.08044863,  0.1003166 , -1.58519423, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.33726852, -0.48299432, -1.59578435, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-1.04376493,  1.21832428,  1.09397975, ...,  1.        ,
         0.        ,  0.        ],
       [-1.36960251, -0.77809567, -0.36679966, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46674041, -0.77976293,  0.11221189, ...,  1.        ,
         0.        ,  0.        ]])